In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, LSTM, GRU, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import sequence
from keras.utils import np_utils
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, classification_report
import csv
import codecs

Using TensorFlow backend.


In [2]:
def verbatim_to_char_seq(name, char_indices, maxLen):
    name_chars = list(name)
    name_chars_indices = list(map(lambda char: char_indices[char], name_chars))
    return sequence.pad_sequences([name_chars_indices], maxLen)[0]

In [3]:
#import verbatims, put into two lists
verbatims = []
labels = []
allowedChars = []
with codecs.open('Data/Catagorization training data.csv', 'r', encoding='ascii', errors='ignore') as f:
    reader = csv.reader(f)
    for line in reader:
        temp = line[1].replace('\n', ' ').replace('\r', ' ').replace('<', ' ').replace('>', ' ').replace('*', ' ')
        temp = temp.replace('%', ' ').replace('&', ' ').replace('#', ' ').replace('~', ' ').replace('@', ' ')
        temp = temp.replace('=', ' ').replace('`', ' ').replace(';', ' ').replace('_', ' ').replace('+', ' ')
        temp = (temp[:198] + '..') if len(temp) > 200 else temp
        verbatims.append(temp)
        labels.append(line[0])

In [4]:
char_list = list(set(''.join(verbatims)))
labels_list = list(set(labels))
char_indices = dict((c, i) for i, c in enumerate(char_list))
indices_char = dict((i, c) for i, c in enumerate(char_list))
label_indices = dict((l, i) for i, l in enumerate(labels_list))
indices_label = dict((i, l) for i, l in enumerate(labels_list))

In [5]:
# get rid of duplicates and blanks

objs = []
for obj in list(zip(verbatims, labels)):
    if len(obj[0].strip()) != 0:
        objs.append(obj)

objs = list(set(objs))

In [6]:
#separate out verbatims and labels again
verbatims = []
labels = []

for n, l in objs:
    verbatims.append(n)
    labels.append(l)
    
print(len(verbatims))

118641


In [7]:
#determine the maximum length of the verbatims
maxLen = 0
for v in verbatims:
    if len(v) > maxLen:
        maxLen = len(v)
print(maxLen)

#if the max length is < 50, pad verbatim
#if maxLen < 50:
#    maxLen = 50

200


In [8]:
#create actual dataset to be fed into keras model
X = []
y = []

for n, l in zip(verbatims, labels):
    X.append(verbatim_to_char_seq(n, char_indices, maxLen))
    y.append(label_indices[l])
    
X = np.array(X).astype(np.uint8)
y=np.array(y)
y = np_utils.to_categorical(np.array(y)).astype(np.bool)

print(X.shape, y.shape)

(118641, 200) (118641, 119)


In [9]:
#create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)
#y = np_utils.to_categorical(y).astype(np.bool)

In [10]:
#create keras model
batch_size = 512
nb_epoch = 200

model = Sequential()
model.add(Embedding(len(char_list), 64, input_length=maxLen, mask_zero=True))
model.add(Bidirectional(LSTM(64, activation="tanh", return_sequences=False, kernel_initializer="glorot_uniform", recurrent_initializer="orthogonal", recurrent_activation="hard_sigmoid")))
model.add(Dropout(0.5))
model.add(Dense(len(labels_list)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

early_stopping = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint(filepath='char_lstm_keras_weights.hdf5', verbose=1, save_best_only=True)
#model.load_weights('char_lstm_keras_weights.hdf5')

In [11]:
#train model on test data
model.fit(X_train, y_train, 
          batch_size=batch_size, 
          epochs=nb_epoch,
          verbose=1,
          shuffle=True,
          validation_split=0.1,
          callbacks=[early_stopping, checkpointer])

Train on 80082 samples, validate on 8898 samples
Epoch 1/200
80082/80082 [==============================] - 165s - loss: 4.3410 - val_loss: 3.9522
Epoch 2/200
80082/80082 [==============================] - 163s - loss: 3.6965 - val_loss: 3.3565
Epoch 3/200
80082/80082 [==============================] - 162s - loss: 3.2389 - val_loss: 2.9377
Epoch 4/200
80082/80082 [==============================] - 162s - loss: 2.8843 - val_loss: 2.6089
Epoch 5/200
80082/80082 [==============================] - 162s - loss: 2.5978 - val_loss: 2.3082
Epoch 6/200
80082/80082 [==============================] - 162s - loss: 2.3593 - val_loss: 2.1543
Epoch 7/200
80082/80082 [==============================] - 163s - loss: 2.1872 - val_loss: 1.9558
Epoch 8/200
80082/80082 [==============================] - 163s - loss: 2.0252 - val_loss: 1.7923
Epoch 9/200
80082/80082 [==============================] - 163s - loss: 1.8975 - val_loss: 1.6821
Epoch 10/200
80082/80082 [==============================] - 163s - lo

KeyboardInterrupt: 

In [ ]:
model.load_weights('char_lstm_keras_weights.hdf5')
preds = model.predict_classes(X_test, batch_size=64, verbose=0)

print('')
print(classification_report(np.argmax(y_test, axis=1), preds, target_names=labels_list))
print('')
print(confusion_matrix(np.argmax(y_test, axis=1), preds))

In [ ]:
import pandas as pd
df=pd.DataFrame(classification_report(np.argmax(y_test, axis=1), preds, target_names=labels_list))

In [ ]:
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
cm=confusion_matrix(np.argmax(y_test, axis=1), preds)
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.colorbar()
tick_marks = np.arange(len(labels_list))
plt.xticks(tick_marks, labels_list, rotation=45)
plt.yticks(tick_marks, labels_list)
thresh = cm.max() / 2.
#for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#    plt.text(j, i, cm[i, j],horizontalalignment="center",color="white" if cm[i, j] > thresh else "black")

#plt.tight_layout()
print(thresh)
plt.ylabel('True label')
plt.xlabel('Predicted label')

In [18]:
#create keras model
batch_size = 512
nb_epoch = 200

model2 = Sequential()
model2.add(Embedding(len(char_list), 64, input_length=maxLen, mask_zero=True))
model2.add(Bidirectional(LSTM(64, activation="tanh", return_sequences=True, kernel_initializer="glorot_uniform", recurrent_initializer="orthogonal", recurrent_activation="hard_sigmoid")))
#model2.add(Dropout(0.5))
model2.add(LSTM(64, activation="tanh", return_sequences=False, kernel_initializer="glorot_uniform", recurrent_initializer="orthogonal", recurrent_activation="hard_sigmoid"))
model2.add(Dropout(0.5))
model2.add(Dense(len(labels_list)))
model2.add(Activation('softmax'))

model2.compile(loss='categorical_crossentropy', optimizer='adam')

early_stopping = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint(filepath='char_lstm2_keras_weights.hdf5', verbose=1, save_best_only=True)
#model2.load_weights('char_lstm2_keras_weights.hdf5')


In [19]:
#train model on test data
model2.fit(X_train, y_train, 
          batch_size=batch_size, 
          epochs=nb_epoch,
          verbose=1,
          shuffle=True,
          validation_split=0.1,
          callbacks=[early_stopping, checkpointer])

Train on 80082 samples, validate on 8898 samples
Epoch 1/200
80082/80082 [==============================] - 284s - loss: 4.4489 - val_loss: 4.2404
Epoch 2/200
80082/80082 [==============================] - 284s - loss: 4.0786 - val_loss: 3.8292
Epoch 3/200
80082/80082 [==============================] - 283s - loss: 3.7422 - val_loss: 3.5141
Epoch 4/200
80082/80082 [==============================] - 284s - loss: 3.4445 - val_loss: 3.1998
Epoch 5/200
80082/80082 [==============================] - 284s - loss: 3.1297 - val_loss: 2.8390
Epoch 6/200
80082/80082 [==============================] - 283s - loss: 2.7933 - val_loss: 2.5298
Epoch 7/200
80082/80082 [==============================] - 283s - loss: 2.4990 - val_loss: 2.2174
Epoch 8/200
80082/80082 [==============================] - 282s - loss: 2.2473 - val_loss: 1.9972
Epoch 9/200
80082/80082 [==============================] - 282s - loss: 2.0534 - val_loss: 1.8002
Epoch 10/200
80082/80082 [==============================] - 282s - lo

80082/80082 [==============================] - 284s - loss: 0.7571 - val_loss: 0.7509
Epoch 32/200
80082/80082 [==============================] - 284s - loss: 0.7457 - val_loss: 0.7204
Epoch 33/200
80082/80082 [==============================] - 284s - loss: 0.7229 - val_loss: 0.7255
Epoch 34/200
80082/80082 [==============================] - 284s - loss: 0.7030 - val_loss: 0.7027
Epoch 35/200
80082/80082 [==============================] - 284s - loss: 0.6913 - val_loss: 0.6982
Epoch 36/200
80082/80082 [==============================] - 284s - loss: 0.6817 - val_loss: 0.6879
Epoch 37/200
80082/80082 [==============================] - 284s - loss: 0.6598 - val_loss: 0.6739
Epoch 38/200
80082/80082 [==============================] - 284s - loss: 0.6450 - val_loss: 0.6664
Epoch 39/200
80082/80082 [==============================] - 284s - loss: 0.6225 - val_loss: 0.6555
Epoch 40/200
80082/80082 [==============================] - 284s - loss: 0.6159 - val_loss: 0.6588
Epoch 41/200
80082/8008

80082/80082 [==============================] - 284s - loss: 0.4013 - val_loss: 0.5857
Epoch 65/200
80082/80082 [==============================] - 284s - loss: 0.4067 - val_loss: 0.5878
Epoch 66/200
80082/80082 [==============================] - 284s - loss: 0.4006 - val_loss: 0.5869
Epoch 67/200
80082/80082 [==============================] - 284s - loss: 0.3944 - val_loss: 0.5870
Epoch 68/200
80082/80082 [==============================] - 284s - loss: 0.4002 - val_loss: 0.5750
Epoch 69/200
80082/80082 [==============================] - 285s - loss: 0.3769 - val_loss: 0.5716
Epoch 70/200
80082/80082 [==============================] - 284s - loss: 0.3658 - val_loss: 0.5873
Epoch 71/200
80082/80082 [==============================] - 284s - loss: 0.3747 - val_loss: 0.5798
Epoch 72/200
80082/80082 [==============================] - 284s - loss: 0.3779 - val_loss: 0.5975
Epoch 73/200
80082/80082 [==============================] - 284s - loss: 0.3747 - val_loss: 0.5802
Epoch 74/200
80082/8008

In [20]:
model2.load_weights('char_lstm2_keras_weights.hdf5')
preds = model2.predict_classes(X_test, batch_size=64, verbose=0)

print('')
print(classification_report(np.argmax(y_test, axis=1), preds, target_names=labels_list))
#print('')
#print(confusion_matrix(np.argmax(y_test, axis=1), preds))


                                       precision    recall  f1-score   support

          General Dynamic Performance       0.79      0.84      0.81       551
           General Rear Entertainment       0.71      0.86      0.78        83
                      Rear Visibility       0.90      0.90      0.90       281
    Tilt / Telescoping Steering Wheel       0.81      0.79      0.80        68
             Environmentally Friendly       0.78      0.76      0.77       134
                        General Seats       0.72      0.80      0.76       506
                           Headlights       0.94      0.98      0.96       395
         Appearance of Steering Wheel       0.33      0.19      0.24        21
          Defrost / Defog Performance       0.94      0.89      0.91       164
                         Fuel Economy       0.96      0.98      0.97      1464
             General Exterior Styling       0.86      0.80      0.83       172
                       General Access       0.83  

In [21]:
#create keras model
batch_size = 512
nb_epoch = 200

model3 = Sequential()
model3.add(Embedding(len(char_list), 64, input_length=maxLen, mask_zero=True))
model3.add(Bidirectional(LSTM(64, activation="tanh", return_sequences=True, kernel_initializer="glorot_uniform", recurrent_initializer="orthogonal", recurrent_activation="hard_sigmoid")))
#model2.add(Dropout(0.5))
model3.add(LSTM(64, activation="tanh", return_sequences=True, kernel_initializer="glorot_uniform", recurrent_initializer="orthogonal", recurrent_activation="hard_sigmoid"))
#model3.add(Dropout(0.5))
model3.add(LSTM(64, activation="tanh", return_sequences=False, kernel_initializer="glorot_uniform", recurrent_initializer="orthogonal", recurrent_activation="hard_sigmoid"))
model3.add(Dropout(0.5))
model3.add(Dense(len(labels_list)))
model3.add(Activation('softmax'))

model3.compile(loss='categorical_crossentropy', optimizer='adam')

early_stopping = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint(filepath='char_lstm3_keras_weights.hdf5', verbose=1, save_best_only=True)
#model2.load_weights('char_lstm3_keras_weights.hdf5')

In [22]:
#train model on test data
model3.fit(X_train, y_train, 
          batch_size=batch_size, 
          epochs=nb_epoch,
          verbose=1,
          shuffle=True,
          validation_split=0.1,
          callbacks=[early_stopping, checkpointer])

Train on 80082 samples, validate on 8898 samples
Epoch 1/200
80082/80082 [==============================] - 404s - loss: 4.4620 - val_loss: 4.3148
Epoch 2/200
80082/80082 [==============================] - 401s - loss: 4.1909 - val_loss: 4.0418
Epoch 3/200
80082/80082 [==============================] - 401s - loss: 4.0280 - val_loss: 3.8984
Epoch 4/200
80082/80082 [==============================] - 401s - loss: 3.8488 - val_loss: 3.6800
Epoch 5/200
80082/80082 [==============================] - 403s - loss: 3.5433 - val_loss: 3.2564
Epoch 6/200
80082/80082 [==============================] - 402s - loss: 3.1581 - val_loss: 2.8901
Epoch 7/200
80082/80082 [==============================] - 402s - loss: 2.8552 - val_loss: 2.5947
Epoch 8/200
80082/80082 [==============================] - 402s - loss: 2.6169 - val_loss: 2.3437
Epoch 9/200
80082/80082 [==============================] - 402s - loss: 2.3565 - val_loss: 2.1223
Epoch 10/200
80082/80082 [==============================] - 402s - lo

80082/80082 [==============================] - 403s - loss: 0.7561 - val_loss: 0.7423
Epoch 32/200
80082/80082 [==============================] - 403s - loss: 0.7350 - val_loss: 0.7416
Epoch 33/200
80082/80082 [==============================] - 402s - loss: 0.7116 - val_loss: 0.7286
Epoch 34/200
80082/80082 [==============================] - 403s - loss: 0.6845 - val_loss: 0.7088
Epoch 35/200
80082/80082 [==============================] - 403s - loss: 0.6661 - val_loss: 0.7161
Epoch 36/200
80082/80082 [==============================] - 403s - loss: 0.6534 - val_loss: 0.6997
Epoch 37/200
80082/80082 [==============================] - 402s - loss: 0.6337 - val_loss: 0.6849
Epoch 38/200
80082/80082 [==============================] - 403s - loss: 0.6192 - val_loss: 0.6759
Epoch 39/200
80082/80082 [==============================] - 404s - loss: 0.6034 - val_loss: 0.6665
Epoch 40/200
80082/80082 [==============================] - 403s - loss: 0.6102 - val_loss: 0.6639
Epoch 41/200
80082/8008

80082/80082 [==============================] - 402s - loss: 0.3575 - val_loss: 0.5993
Epoch 66/200
80082/80082 [==============================] - 402s - loss: 0.3393 - val_loss: 0.6124
Epoch 67/200
80082/80082 [==============================] - 402s - loss: 0.3394 - val_loss: 0.6194
Epoch 68/200
80082/80082 [==============================] - 402s - loss: 0.3345 - val_loss: 0.6002
Epoch 69/200
80082/80082 [==============================] - 403s - loss: 0.3312 - val_loss: 0.6114
Epoch 70/200
80082/80082 [==============================] - 404s - loss: 0.3225 - val_loss: 0.6152
Epoch 71/200
 7680/80082 [=>............................] - ETA: 351s - loss: 0.3171

KeyboardInterrupt: 

In [ ]:
model3.load_weights('char_lstm3_keras_weights.hdf5')
preds = model2.predict_classes(X_test, batch_size=64, verbose=0)

print('')
print(classification_report(np.argmax(y_test, axis=1), preds, target_names=labels_list))
#print('')
#print(confusion_matrix(np.argmax(y_test, axis=1), preds))

In [ ]:
v=[]
with codecs.open('Data/verbatims.csv', 'r', encoding='ascii', errors='ignore') as f:
    reader = csv.reader(f)
    for line in reader:
        v.append(line[0])

In [ ]:
v[1]

In [ ]:
X_real=[]
vo=map(lambda x:x.upper().replace("'","").replace("\"","").replace("?","").replace("~",""),v)
for n in vo:
    X_real.append(verbatim_to_char_seq(n, char_indices, maxLen))
    
X_real = np.array(X_real).astype(np.uint8)

In [ ]:
p = model2.predict_classes(X_real, batch_size=64, verbose=0)

In [ ]:
pArray = map(lambda x:indices_label[x],p)

In [ ]:
pArray

In [ ]:
with codecs.open('Data/verbatimResults.csv', 'w', encoding='ascii', errors='ignore') as f:
    wr = csv.writer(f, dialect='excel')
    for item in pArray:
        wr.writerow([item])